In [39]:
!pip install flask mysql-connector-python pandas

In [40]:
import mysql.connector

def run_query(query, params=None):
    conn = mysql.connector.connect(
        host='localhost',
        user='root',
        password='coconuts',
        database='StoreData'
    )
    cursor = conn.cursor(dictionary=True)
    cursor.execute(query, params or ())
    if query.strip().lower().startswith("select"):
        result = cursor.fetchall()
    else:
        conn.commit()
        result = {"message": "Query executed successfully"}
    cursor.close()
    conn.close()
    return result

In [41]:
# Create Database
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='coconuts'
)
cursor = conn.cursor()
cursor.execute("CREATE DATABASE IF NOT EXISTS StoreData;")
conn.commit()
cursor.close()
conn.close()

In [42]:
# Create Table
run_query("""
CREATE TABLE IF NOT EXISTS Sales (
    id INT PRIMARY KEY,
    store_id VARCHAR(10),
    total_sales DECIMAL(10,2),
    date DATE
)
""")

{'message': 'Query executed successfully'}

In [44]:
# STEP 3 — View existing IDs and insert a new sale record

# View existing IDs so we avoid duplicates
existing_ids = run_query("SELECT id FROM Sales")
print("Existing IDs:", [row["id"] for row in existing_ids])

# Now insert a record using a new ID
new_data = {
    "id": 1008,  # ✅ Make sure this number is not already listed above
    "store_id": "TX009",
    "total_sales": 199.99,
    "date": "2023-04-08"
}

try:
    response = run_query("""
    INSERT INTO Sales (id, store_id, total_sales, date)
    VALUES (%s, %s, %s, %s)
    """, (new_data["id"], new_data["store_id"], new_data["total_sales"], new_data["date"]))
    print("Insert response:", response)
except Exception as e:
    print("⚠️ Error inserting record:", e)



Existing IDs: [1001, 1002, 1003, 1007]
Insert response: {'message': 'Query executed successfully'}


In [46]:
# Fetch Records Between Dates
import pandas as pd

start_date = "2023-01-01"
end_date = "2023-12-31"

result = run_query("""
SELECT * FROM Sales WHERE date BETWEEN %s AND %s
""", (start_date, end_date))

pd.DataFrame(result)

,id,store_id,total_sales,date
0,1001,TX001,150.75,2023-01-20
1,1002,TX001,150.75,2023-01-20
2,1003,TX002,220.45,2023-02-15
3,1007,TX001,150.75,2023-01-20
4,1008,TX009,199.99,2023-04-08
